# Script para planilha de lista eleitoral

In [13]:
import pypdf
import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns   

from pathlib import Path
from functools import partial
from langchain_openai import  AzureChatOpenAI
from langchain.prompts import PromptTemplate
from utils_ccd import get_connection, extract_text_from_pdf
from pathlib import Path

DIR_INFORMACOES = '/media/informacoes_pdf/'
def get_info_file_path(row):
    return Path(DIR_INFORMACOES) / row['setor'].strip() / row['arquivo']

conn = get_connection()

In [2]:
df_lista = pd.read_excel("docs/lista_eleitoral_250527.xlsx")
processos = ','.join(map(lambda x: f"'{x}'", df_lista['NUM_PROCESSO']))

sql_processo = f''' 
SELECT *
FROM processo.dbo.Processos p
WHERE CONCAT(p.numero_processo, '/', p.ano_processo) IN ({processos});
'''
df_processo = pd.read_sql(sql_processo, conn)

sql_all_informacoes_processos = f'''
    SELECT concat(rtrim(inf.setor),'_',inf.numero_processo ,'_',inf.ano_processo,'_',RIGHT(concat('0000',inf.ordem),4),'.pdf') as arquivo,
    ppe.SequencialProcessoEvento as evento,
    CONCAT(inf.numero_processo,'/', inf.ano_processo) as processo,
    inf.*
    FROM processo.dbo.vw_ata_informacao inf INNER JOIN processo.dbo.Pro_ProcessoEvento ppe 
        ON inf.idinformacao = ppe.idinformacao
    WHERE concat(inf.numero_processo, '/', inf.ano_processo) in ({processos})
'''
all_informacoes_processos = pd.read_sql(sql_all_informacoes_processos, conn)

def filter_infos(df):
    df = df.copy()
    df = df[~df['resumo'].str.contains('capa', case=False, na=False)]
    df = df[~df['resumo'].str.contains('digitalizado', case=False, na=False)]
    return df


/tmp/ipykernel_18255/1476759274.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_processo = pd.read_sql(sql_processo, conn)
/tmp/ipykernel_18255/1476759274.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  all_informacoes_processos = pd.read_sql(sql_all_informacoes_processos, conn)


In [27]:
all_informacoes_processos = filter_infos(all_informacoes_processos)

In [ ]:
all_informacoes_processos['caminho_arquivo'] = all_informacoes_processos.apply(get_info_file_path, axis=1)

In [ ]:
all_informacoes_processos['texto'] = all_informacoes_processos['caminho_arquivo'].apply(extract_text_from_pdf)

In [3]:
llm = AzureChatOpenAI(model_name="gpt-4o")
def is_parecer_previo(texto):
    try:
        prompt = PromptTemplate.from_template("""
        Você é um agente que analisa e resume informações de processos do Tribunal de Contas do Estado do Rio Grande do Norte.

        O texto da informação é o seguinte:
        "{input}"
                                              
        Defina se a informação é um parecer prévio ou não.
        Responda apenas com "Sim" ou "Não". Não forneça explicações ou justificativas.                                           

        Sua resposta:
        """)

        chain = prompt | llm
        msg_obj = chain.invoke(texto)
        return msg_obj.content
    except Exception as e:
        print(f"Error processing {texto}: {e}")
        return None

In [26]:
import re

def is_parecer_previo_regex(texto):
    try:
        # Regex pattern to match "parecer prévio" or similar phrases
        pattern = r'\b(parecer prévio|parecer prévio do tribunal|parecer prévio do tce)\b'
        if re.search(pattern, texto, re.IGNORECASE):
            return 'Sim'
        else:
            return 'Não'
    except Exception as e:
        print(f"Error processing {texto}: {e}")
        return None

In [29]:
all_informacoes_processos['parecer_previo'] = all_informacoes_processos['texto'].apply(is_parecer_previo_regex)

In [31]:
all_informacoes_processos['parecer_previo'].value_counts()

Não    10547
Sim      275
Name: parecer_previo, dtype: int64